In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import backend as k
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.preprocessing.image import load_img
import itertools
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.models import load_model
model=load_model ('classification_of_cable_with_mobilenetV2_last.h5')

In [ ]:
#RANDOM

In [ ]:
test_path = 'F:/Data/ORIG/Test2'

In [ ]:
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path,target_size=(75,75),classes=['cable','no'],batch_size = 10,shuffle = False)

In [ ]:
test_labels = test_batches.classes

In [ ]:

start = time.perf_counter()
predictions = model.predict_generator(test_batches,steps=20,verbose=0)
elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)


In [ ]:
cm = confusion_matrix(test_labels,predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
#Evaluation of Model - Confusion Matrix Plot
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [ ]:
cm_plot_labels = ['Cable','No Cable']
plot_confusion_matrix(cm,cm_plot_labels,title= 'Confusion Matrix for Randomly Imges')

In [ ]:
ground_truth = test_batches.classes
label2index = test_batches.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
predicted_classes = np.argmax(predictions,axis=1)
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),test_batches.samples))

In [ ]:
fnames = test_batches.filenames
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(test_path,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,predictions.argmax(axis=1)))